生成schema_links的prompt

In [7]:
import json

def generate_schema_linking_prompt(json_data, md_file_path):
    """
    根据JSON数据生成Schema Linking的prompt

    Args:
        json_data: 包含问题、表列表和知识的JSON数据
        md_file_path: MD文件路径

    Returns:
        生成的schema linking prompt字符串
    """
    try:
        # 解析JSON数据
        if isinstance(json_data, str):
            data = json.loads(json_data)
        else:
            data = json_data

        # 提取关键信息
        question = data.get("question", "")
        table_list = data.get("table_list", [])
        knowledge = data.get("knowledge", "")

        # 读取MD文件内容
        with open(md_file_path, 'r', encoding='utf-8') as file:
            md_content = file.read()

        # 收集所有表的完整schema
        schema_parts = []
        for table_name in table_list:
            schema = get_table_schema(md_content, table_name)
            if schema:
                schema_parts.append(f"表 {table_name} 的完整结构:\n{schema}")

        # 构建schema linking的prompt
        prompt_parts = []

        # 1. 问题描述
        prompt_parts.append(f"问题: {question}")

        # 2. 所有表的完整schema
        if schema_parts:
            prompt_parts.append("\n相关表的完整结构:")
            prompt_parts.extend(schema_parts)

        # 3. 业务知识
        if knowledge:
            prompt_parts.append(f"\n业务知识:\n{knowledge}")

        # 4. Schema Linking任务说明
        prompt_parts.append("\n任务说明:")
        prompt_parts.append("请分析上述问题，从提供的表结构中找出解决问题所需的所有列。")
        prompt_parts.append("返回格式要求:")
        prompt_parts.append("- 以列表形式返回，每个元素为 table_name.column_name")
        prompt_parts.append("- 只返回列名列表，不要返回其他内容")
        prompt_parts.append("- 确保表名和列名与schema中完全一致")
        prompt_parts.append("\n请返回需要的列列表:")

        return "\n".join(prompt_parts)

    except Exception as e:
        return f"生成schema linking prompt时出错: {e}"


def get_table_schema(md_content, table_name):
    """
    从MD内容中提取指定表的schema

    Args:
        md_content: MD文件内容
        table_name: 表名

    Returns:
        表的schema字符串
    """
    lines = md_content.split('\n')
    schema_lines = []
    in_target_table = False
    table_found = False

    for line in lines:
        # 检查是否找到目标表
        if f"# Table: {table_name}" in line:
            in_target_table = True
            table_found = True
            schema_lines.append(line)
            continue

        # 如果已经在目标表中，检查是否遇到下一个表
        if in_target_table:
            if line.startswith("# Table:"):
                # 遇到下一个表，结束当前表
                break
            else:
                # 添加当前表的内容
                schema_lines.append(line)

    if not table_found:
        return f"未找到表 {table_name} 的结构"

    return '\n'.join(schema_lines)


def load_json_data(json_file_path):
    """
    从JSON文件加载数据

    Args:
        json_file_path: JSON文件路径

    Returns:
        JSON数据列表
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 检查数据格式：可能是列表或字典
        if isinstance(data, list):
            return data
        elif isinstance(data, dict):
            # 如果JSON文件是单个对象，包装成列表
            return [data]
        else:
            print(f"警告: JSON文件格式不支持: {type(data)}")
            return []
    except Exception as e:
        print(f"加载JSON文件时出错: {e}")
        return []


def generate_all_schema_linking_prompts(json_file_path, md_file_path, output_file_path):
    """
    从JSON文件读取所有数据并生成schema linking prompts

    Args:
        json_file_path: JSON数据文件路径
        md_file_path: MD文件路径
        output_file_path: 输出文件路径
    """
    # 加载JSON数据
    json_data_list = load_json_data(json_file_path)

    if not json_data_list:
        print("未加载到任何JSON数据，请检查文件路径和格式")
        return

    print(f"成功加载 {len(json_data_list)} 条JSON数据")

    # 生成所有prompts
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for i, json_data in enumerate(json_data_list):
            print(f"正在生成第 {i+1} 个prompt...")

            prompt = generate_schema_linking_prompt(json_data, md_file_path)

            # 写入分隔符和prompt
            output_file.write(f"=== Prompt {i+1} ===\n")
            output_file.write(f"SQL ID: {json_data.get('sql_id', 'N/A')}\n")
            output_file.write(prompt)
            output_file.write("\n\n" + "="*80 + "\n\n")

        print(f"已完成! 所有prompts已保存到: {output_file_path}")


# 直接调用批量生成函数
json_file_path = "../data/final_dataset.json"  # 替换为您的JSON文件路径
md_file_path = "../data/final_algorithm_competition_schema.md"    # 替换为您的MD文件路径
output_file_path = "../data/all_schema_linking_prompts.txt"  # 输出文件路径

generate_all_schema_linking_prompts(json_file_path, md_file_path, output_file_path)

成功加载 101 条JSON数据
正在生成第 1 个prompt...
正在生成第 2 个prompt...
正在生成第 3 个prompt...
正在生成第 4 个prompt...
正在生成第 5 个prompt...
正在生成第 6 个prompt...
正在生成第 7 个prompt...
正在生成第 8 个prompt...
正在生成第 9 个prompt...
正在生成第 10 个prompt...
正在生成第 11 个prompt...
正在生成第 12 个prompt...
正在生成第 13 个prompt...
正在生成第 14 个prompt...
正在生成第 15 个prompt...
正在生成第 16 个prompt...
正在生成第 17 个prompt...
正在生成第 18 个prompt...
正在生成第 19 个prompt...
正在生成第 20 个prompt...
正在生成第 21 个prompt...
正在生成第 22 个prompt...
正在生成第 23 个prompt...
正在生成第 24 个prompt...
正在生成第 25 个prompt...
正在生成第 26 个prompt...
正在生成第 27 个prompt...
正在生成第 28 个prompt...
正在生成第 29 个prompt...
正在生成第 30 个prompt...
正在生成第 31 个prompt...
正在生成第 32 个prompt...
正在生成第 33 个prompt...
正在生成第 34 个prompt...
正在生成第 35 个prompt...
正在生成第 36 个prompt...
正在生成第 37 个prompt...
正在生成第 38 个prompt...
正在生成第 39 个prompt...
正在生成第 40 个prompt...
正在生成第 41 个prompt...
正在生成第 42 个prompt...
正在生成第 43 个prompt...
正在生成第 44 个prompt...
正在生成第 45 个prompt...
正在生成第 46 个prompt...
正在生成第 47 个prompt...
正在生成第 48 个prompt...
正在生成第 49 个prompt...
正在生成第 50 个pr